First lets do all the imports and make sure the environment has everything

In [25]:
import sys
import random
import dtk_hiv_intrahost as gi
import dtk_hivdelayed_iv as delay
import dtk_artbasic_iv as art
import dtk_malecirc_iv as vmmc
import dtk_modcoinf_iv as coinf
from hiv_person import HivPerson

Second, let's set some variables and create our callbacks (which are just functions of ours the system calls). The "publish_cb" is the key one that defines the behaviour of the cascade. The others can be ignored.

In [26]:
ignore_signals = [ "EveryUpdate", "NewInfectionEvent", "HappyBirthday" ]

pct_immedate_art = 0.5

def subscribe_cb( signal, person ):
    print( "{person} wants to subscribe to {signal}" )

def unsubscribe_cb( signal, person ):
    print( "{person} wants to unsubscribe from {signal}" )

def publish_cb( signal, person ):
    if signal in ignore_signals:
        return
    p = HivPerson.get_person_from_id( person )
    print( f"SIGNAL: Person:{person}\tAge:{p.age():0.2f}\tSignal:{signal}" )

    if signal=="STINewInfection":
        # STINewInfection --> HIVNewlyDiagnosed (see DelayedIntervention.json for config)
        print( "ACTION: Give Delay to Newly Infected." )
        p.give_intervention( delay )
    elif signal=="HIVNewlyDiagnosed":
        # Some % go on ART
        if random.random() < pct_immedate_art:
            print( "ACTION: Give ART after delay." )
            p.give_intervention( art )
        else:
        # Other 90% (of males) get VMMC
            print( "ACTION: Give VMMC after delay." )
            sex = p.sex()
            if sex == 0: # sorry ladies, this one's only for the boys
                print( "...if male... (TBD)" )
                p.give_intervention( vmmc )
            else:
                print( "...unless female..." )
    elif signal=="STIDebut":
        print( "ACTION: Give STI co-infection upon debut." )
        import dtk_hivdelayed_iv as sti_delay
        sti_delay.set_name( "STD" )
        p.give_intervention( sti_delay )
    elif signal=="STIDebut2":
        p.give_intervention( coinf )
    elif signal=="DiseaseDeaths":
        print( f"Somebody died of HIV at age {p.age():0.2f}." )

gi.set_sub_cb( subscribe_cb )
gi.set_unsub_cb( unsubscribe_cb )
gi.set_pub_cb( publish_cb )

Now we define a function that we are going to use for each person we create. It infects them, and then "runs them forward" until they die. It tracks their CD4 so that can be plotted, or otherwise analyzer, which we are not doing yet.

In [27]:
def infect_and_run( person ):
    """
    Infect person and update them until they die of HIV or an update limit is reached.
    Collect CD4 counts for optional plotting.
    """
    cd4s = []
    person.force_infect()
    updates = 0
    while person.dead() == False and updates < 10000:
        person.update()
        cd4 = person.cd4()
        cd4s.append( cd4 )
        updates += 1
        #is_dead = person.dead()
        #is_infected = person.infected()
        #print( "Is infected? {0} CD4 {1}, Dead? {2}.".format( is_infected, cd4, is_dead  ) )
    if person.dead() == False:
        print( "We kept them alive." )
    else:
        print( "They died." )
    return cd4s

Everything is set up. Now create some people, infect them, and update them until death.

In [28]:
for _ in range(100): # Test 100 separate people
    # Create person with random age (0-100) and random sex.
    # Note that there are no relationships here. Just people.
    person = HivPerson( age_in_years=random.uniform( 0, 100 ) )
    infect_and_run( person )
    print( "" ) # separator

Created new person with age 92.54 and sex male
SIGNAL: Person:701	Age:92.54	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:701	Age:92.79	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:701	Age:99.61	Signal:STIDebut2
SIGNAL: Person:701	Age:103.20	Signal:HIVSymptomatic
SIGNAL: Person:701	Age:103.68	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:701	Age:104.44	Signal:DiseaseDeaths
Somebody died of HIV at age 104.44.
They died.

Created new person with age 62.97 and sex female
SIGNAL: Person:702	Age:62.97	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:702	Age:63.21	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:702	Age:67.83	Signal:HIVSymptomatic
SIGNAL: Person:702	Age:68.51	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:702	Age:69.05	Signal:STIDebut2
SIGNAL: Person:702	Age:69.26	Signal:DiseaseDeaths
Somebody died of HIV at age 69.26.
They died.

Created new person with age 49.39 and sex male
SIGNAL: Person:703	Ag

SIGNAL: Person:722	Age:37.29	Signal:HIVSymptomatic
SIGNAL: Person:722	Age:38.50	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:722	Age:39.25	Signal:DiseaseDeaths
Somebody died of HIV at age 39.25.
They died.

Created new person with age 17.95 and sex male
SIGNAL: Person:723	Age:17.95	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:723	Age:18.19	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:723	Age:22.38	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:723	Age:22.41	Signal:HIVSymptomatic
SIGNAL: Person:723	Age:23.13	Signal:DiseaseDeaths
Somebody died of HIV at age 23.13.
They died.

Created new person with age 92.71 and sex female
SIGNAL: Person:724	Age:92.71	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:724	Age:92.96	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:724	Age:93.08	Signal:STIDebut2
SIGNAL: Person:724	Age:98.18	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:724	Age:98.27	Signal:HIVSymptomatic
SI

They died.

Created new person with age 41.12 and sex male
SIGNAL: Person:744	Age:41.12	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:744	Age:41.36	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:744	Age:49.04	Signal:HIVSymptomatic
SIGNAL: Person:744	Age:50.78	Signal:STIDebut2
SIGNAL: Person:744	Age:51.04	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:744	Age:51.79	Signal:DiseaseDeaths
Somebody died of HIV at age 51.79.
They died.

Created new person with age 76.04 and sex female
SIGNAL: Person:745	Age:76.04	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:745	Age:76.28	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:745	Age:76.72	Signal:HIVSymptomatic
SIGNAL: Person:745	Age:77.72	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:745	Age:78.47	Signal:DiseaseDeaths
Somebody died of HIV at age 78.47.
They died.

Created new person with age 81.02 and sex female
SIGNAL: Person:746	Age:81.02	Signal:STINewInfection
ACTIO

SIGNAL: Person:762	Age:53.61	Signal:HIVSymptomatic
SIGNAL: Person:762	Age:53.81	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:762	Age:54.56	Signal:DiseaseDeaths
Somebody died of HIV at age 54.56.
They died.

Created new person with age 6.22 and sex female
SIGNAL: Person:763	Age:6.22	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:763	Age:6.46	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:763	Age:13.00	Signal:STIDebut
ACTION: Give STI co-infection upon debut.
SIGNAL: Person:763	Age:13.68	Signal:STIDebut2
SIGNAL: Person:763	Age:14.42	Signal:STIDebut2
SIGNAL: Person:763	Age:27.76	Signal:HIVSymptomatic
SIGNAL: Person:763	Age:28.45	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:763	Age:29.20	Signal:DiseaseDeaths
Somebody died of HIV at age 29.20.
They died.

Created new person with age 79.62 and sex male
SIGNAL: Person:764	Age:79.62	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:764	Age:79.87	Signal:HIVInfectionStageEn

SIGNAL: Person:797	Age:88.95	Signal:HIVSymptomatic
SIGNAL: Person:797	Age:89.26	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:797	Age:90.01	Signal:DiseaseDeaths
Somebody died of HIV at age 90.01.
They died.

Created new person with age 80.83 and sex male
SIGNAL: Person:798	Age:80.83	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:798	Age:81.08	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:798	Age:85.24	Signal:HIVSymptomatic
SIGNAL: Person:798	Age:86.59	Signal:STIDebut2
SIGNAL: Person:798	Age:87.25	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:798	Age:88.00	Signal:DiseaseDeaths
Somebody died of HIV at age 88.00.
They died.

Created new person with age 64.89 and sex male
SIGNAL: Person:799	Age:64.89	Signal:STINewInfection
ACTION: Give Delay to Newly Infected.
SIGNAL: Person:799	Age:65.14	Signal:HIVInfectionStageEnteredLatent
SIGNAL: Person:799	Age:70.68	Signal:HIVInfectionStageEnteredAIDS
SIGNAL: Person:799	Age:70.82	Signal:HIVSymptomatic
SIGN

Below is some placeholder code to start building a clickable GUI...

In [24]:
import ipywidgets
from IPython.display import display

from IPython.core.display import HTML

def func(btn):
    print('Hi!')

btn1 = ipywidgets.Button(description="Click me!")
btn1.on_click(func)
btn2 = ipywidgets.Button(description="Click me!")
btn2.on_click(func)
display(btn1)
display(btn2)

display(HTML(
        '<table>' +
        '<tr><td>Something here</td></tr>' +
        '<tr><td>Something here</td></tr>' +
        '</table>'
    ))

Button(description='Click me!', style=ButtonStyle())

Button(description='Click me!', style=ButtonStyle())

Something here
Something here
